# Interaction with the World Homework (#3)
Python Computing for Data Science (c) J Bloom, UC Berkeley 2018

Due Tuesday 2pm, Feb 20, 2018

# 1) Monty: The Python Siri

Let's make a Siri-like program (call it Monty!) with the following properties:
   - record your voice command
   - use a webservice to parse that sound file into text
   - based on what the text, take three different types of actions:
       - send an email to yourself
       - do some math
       - tell a joke

So for example, if you say "Monty: email me with subject hello and body goodbye", it will email you with the appropriate subject and body. If you say "Monty: tell me a joke" then it will go to the web and find a joke and print it for you. If you say, "Monty: calculate two times three" it should response with printing the number 6.

Hint: you can use speed-to-text apps like Houndify (or, e.g., Google Speech https://cloud.google.com/speech/) to return the text (but not do the actions). You'll need to sign up for a free API and then follow documentation instructions for using the service within Python. 

In [2]:
import pyaudio
import smtplib
import os
import my_credentials as creds

from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email import encoders
from email.utils import COMMASPACE, formatdate

In [9]:
# taken from lecture, changed print outputs
def MontyListens():
    # set up pyaudio for recording
    chunk = 1024
    FORMAT = pyaudio.paInt32
    CHANNELS = 1
    RATE = 44100
    RECORD_SECONDS = 5
    p = pyaudio.PyAudio()
    stream = p.open(format = FORMAT,
        channels = CHANNELS,
        rate = RATE,
        input = True,
        frames_per_buffer = chunk)
    print("**Monty is listening!")
    # record data
    all = []
    for i in range(0, int(RATE / chunk * RECORD_SECONDS)):
        data = stream.read(chunk)
        all.append(data)
    print("**Monty is thinking...**")
    # stop listening and close pyaudio
    stream.close()
    p.terminate()
    return data

# from the email_example.py code, removed file attachment ability, SMTP -> SMTP_SSL
def MontyEmails(sender, pwd, to, subject, text):
    # create message
    msg = MIMEMultipart()
    msg["From"] = sender
    msg["To"] = COMMASPACE.join(to)
    msg["Date"] = formatdate(localtime=True)
    msg["Subject"] = subject
    msg.attach(MIMEText(text))
    
    # open mail server, login, send message, close server
    mailServer = smtplib.SMTP_SSL("smtp.gmail.com", creds.GMAIL_SMTP_PORT)
    mailServer.login(sender, pwd)
    mailServer.sendmail(sender, to, msg.as_string())
    mailServer.close()

In [10]:
gmail_user = creds.GMAIL_USERNAME
gmail_password = creds.GMAIL_PASSWORD


# MontyEmails(sender=gmail_user, 
#     pwd=gmail_password, 
#     to=[gmail_user,],   # include an extra comma in the "to" list to 
#                                    # account for the COMMASPACE.join(to)
#     subject="Email from Python", 
#     text="Whoooo!\n"
#     )


# 2) Write a program that identifies musical notes from sound (AIFF) files. 

  - Run it on the supplied sound files (12) and report your program’s results. 
  - Use the labeled sounds (4) to make sure it works correctly. The provided sound files contain 1-3 simultaneous notes from different organs.
  - Save copies of any example plots to illustrate how your program works.
  
  https://piazza.com/berkeley/spring2018/ay250class13410/resources -> Homeworks -> hw3_sound_files.zip

Hints: You’ll want to decompose the sound into a frequency power spectrum. Use a Fast Fourier Transform. Be care about “unpacking” the string hexcode into python data structures. The sound files use 32 bit data. Play around with what happens when you convert the string data to other integer sizes, or signed vs unsigned integers. Also, beware of harmonics.